In [1]:
#clone fusion_twas library before running this
!git clone https://github.com/gusevlab/fusion_twas.git

fatal: destination path 'fusion_twas' already exists and is not an empty directory.


In [2]:
import subprocess
import sys
import os
import shutil
import pandas as pd
import numpy as np

In [3]:
def shell_do(command, log=False, return_log=False):
    print(f'Executing: {(" ").join(command.split())}', file=sys.stderr)

    res=subprocess.run(command.split(), stdout=subprocess.PIPE)

    if log:
        print(res.stdout.decode('utf-8'))
    if return_log:
        return(res.stdout.decode('utf-8'))

In [4]:
#set paths
basedir = '/data/songy4/tes'
datadir = f'{basedir}/data_folder'
fusiondir = f'/data/songy4/twas/fusion_twas'
geno_path_case = f'{datadir}/qc_genotypes_tes_case_hg19_lifted'
geno_path_cont = f'{datadir}/qc_genotypes_tes_control_hg19_lifted'
transcript_list_path = f'{datadir}/transcript_list.txt'
te_list_path = f'{datadir}/te_list.txt'
pheno_path_tecase = f'{datadir}/baseline_te_case.txt'
pheno_path_tecont = f'{datadir}/baseline_te_control.txt'
pheno_path_trcase = f'{datadir}/base_transcript_case.txt'
pheno_path_trcont = f'{datadir}/base_transcript_control.txt'
coord_path_te = f'{datadir}/te_coordinate.txt'
coord_path_tr = f'{datadir}/transcript_coordinate.txt'
covar_path_case = f'{datadir}/te_covariates_case.txt'
covar_path_cont = f'{datadir}/te_covariates_control.txt'
gcta = f'{fusiondir}/gcta_nr_robust'
gemma = f'gemma-0.98.3-linux-static'
fusion_ldref_basename = f'{fusiondir}/LDREF/1000G.EUR'
fusion_compute_weights_script = f'{fusiondir}/FUSION.compute_weights.R'

!mkdir --parents {basedir}/output/weights_case
!mkdir --parents {basedir}/output/tmp_case


# Pipeline for TWAS run

In [4]:
# get gene list
##test with couple genes first before running all gene_list
#transcript_list_df = pd.read_csv(transcript_list_path, engine='c',sep='\t')
#transcript_list = list(transcript_list_df.ID)
#transcript_list = ['ENST00000000233.10', 'ENST00000000412.8', 'ENST00000000442.11'] # for testing
# gene_list = ['ENSG00000186092']
#pheno_trcase = pd.read_csv(pheno_path_trcase, engine='c', sep='\t')
#pheno_trcont = pd.read_csv(pheno_path_trcont, engine='c',sep='\t')
#coords = pd.read_csv(coord_path_tr, engine='c',sep='\t')
#covars_case = pd.read_csv(covar_path_case, engine='c',sep='\t')
#covars_cont = pd.read_csv(covar_path_cont, sep='\t')

In [4]:
# now put together pipeline
covars = pd.read_csv(covar_path_case, sep='\t')

for i in range(21 ,23): 
    pheno_i = pd.read_csv(f'{datadir}/base_transcript_case_chr{i}.txt', engine='c')
    coords_i = pd.read_csv(f'{datadir}/transcript_coordinate_chr{i}.txt',  engine='c')
    gene_list_df_i = pd.read_csv(f'{datadir}/transcript_list_chr{i}.txt', engine='c')
    gene_list_i = list(gene_list_df_i.ID)
    
    compweights_swarmfile_i = f'{basedir}/compute_weights_{i}.swarm'

    with open(compweights_swarmfile_i, 'w') as f:
    
        for gene in gene_list_i:
            OUT = f'output/tmp_case/{gene}'
            FINAL_OUT = f'output/weights_case/{gene}'
            #get chr start stop
            _chr = coords_i.loc[coords_i.ID == gene, 'X.Chr'].item()
            _start = coords_i.loc[coords_i.ID == gene, 'start'].item()-0.5e6
            _stop = coords_i.loc[coords_i.ID == gene, 'end'].item()+0.5e6

            if _start < 0:
                _start = 0

            _start = int(_start)
            _stop = int(_stop)

            pheno_i[['FID','IID', gene]].to_csv(f'{OUT}.pheno', sep='\t', header=False, index=False)
    #         pheno[['FID','IID', gene]].to_csv(_phenoname, sep='\t', header=False, index=False)

            plink_cmd_i = f'\
plink --bfile {geno_path_case} \
--pheno {OUT}.pheno \
--keep {OUT}.pheno \
--chr {_chr} \
--from-bp {_start} \
--to-bp {_stop} \
--extract {fusion_ldref_basename}.{_chr}.bim \
--make-bed \
--out {OUT}'  

#        shell_do(plink_cmd, log=True, return_log=True)

            fusion_cmd_i = f'\
Rscript {fusion_compute_weights_script} \
--bfile {OUT} \
--tmp {OUT}.tmp \
--out {FINAL_OUT} \
--PATH_gemma {gemma} \
--PATH_plink plink \
--PATH_gcta {gcta} \
--covar {covar_path_case} \
--verbose 2 \
--save_hsq \
--models top1,lasso,enet'
##include "blup,bslmm" to the --models in fusion_cmd for future     
# 
#        shell_do(fusion_cmd, log=True, return_log=True)

            f.write(f'{plink_cmd_i} && {fusion_cmd_i}\n')
        f.close()

In [9]:
# run swarm
#compweights_swarmfile_3 = f'{basedir}/compute_weights_3.swarm'
#swarm_cmd_3 = f'swarm -f {compweights_swarmfile_3} -g 16 -t 16 --time=10:00:00 --logdir swarm --gres=lscratch:20 --module plink,GEMMA/0.96 --partition=norm'
# shell_do(swarm_cmd)
#print(swarm_cmd_3)
#!{swarm_cmd_3}
##check swarm jobs state in hpc.nih.gov/dashboard/
##check .e, .o, and .log files of swarm jobs in swarm folder (same name as swarm jobid in hpc dashboard)

swarm -f /data/songy4/tes/compute_weights_3.swarm -g 16 -t 16 --time=10:00:00 --logdir swarm --gres=lscratch:20 --module plink,GEMMA/0.96 --partition=norm
15461033


In [5]:
# run swarm
for i in range(21,23): 
    compweights_swarmfile_i = f'{basedir}/compute_weights_{i}.swarm'
    swarm_cmd_i = f'swarm -f {compweights_swarmfile_i} -g 8 -t 2 --time=10:00:00 --logdir swarm --gres=lscratch:20 --module plink,GEMMA/0.96 --partition=norm'
#--devel
# shell_do(swarm_cmd)
    print(swarm_cmd_i)
    !{swarm_cmd_i}
###for this specific project, 8GB memory and 2 CPU are enough###
##check swarm jobs state in hpc.nih.gov/dashboard/
##check .e, .o, and .log files of swarm jobs in swarm folder (same name as swarm jobid in hpc dashboard)

swarm -f /data/songy4/tes/compute_weights_21.swarm -g 16 -t 16 --time=10:00:00 --logdir swarm --gres=lscratch:20 --module plink,GEMMA/0.96 --partition=norm
16844245
swarm -f /data/songy4/tes/compute_weights_22.swarm -g 16 -t 16 --time=10:00:00 --logdir swarm --gres=lscratch:20 --module plink,GEMMA/0.96 --partition=norm
16844251


In [7]:
#check number of weights in output/weights folder 
###run this in terminal

#go to the weights folder
!cd /data/songy4/tes/output/weights_case
#check how many files are in total
! ls | wc -l
#there are 218193
#check how many .RDat file is in weights folder
!ls -lR ./*.RDat | wc -l
#there are 9482 RDat files

61
ls: cannot access ./*.RDat: No such file or directory
0


TWAS

In [24]:
#open gene_chr_start_stop.tab -- this has chr and position info
chr_pos_df = pd.read_csv(coord_path_tr, sep='\t')
#rearrange the columns 
chr_pos_df = chr_pos_df[['ID', 'X.Chr','start','end']]
#change column names in chr_pos_df
chr_pos_df.columns = ['ID', 'CHR', 'P0', 'P1']

chr_pos_df

,ID,CHR,P0,P1
0,ENST00000467231.5,1,90916390,91022014
1,ENST00000359621.5,1,15456732,15472091
2,ENST00000470394.1,1,21233570,21238350
3,ENST00000470395.1,1,54032019,54036802
4,ENST00000470396.1,1,151266308,151266725
...,...,...,...,...
219238,ENST00000447224.5,22,30555938,30560759
219239,ENST00000472551.1,22,44196098,44219533
219240,ENST00000447261.6,22,24632915,24647760
219241,ENST00000426113.5,22,20967248,20974096


In [9]:
#make gene_path.csv
! ls -l ./output/weights_case | grep -E 'ENSG|.RDat' | awk '{print $9", ./weights/"$9}' | sed 's/.wgt.RDat,/,/g' > ./data_folder/gene_path_case.csv

## list files in weights folder, search lines with same pattern as 'ENSG'
## scan file line by line, splits each input line
## substitute .wgt.RDat, with , in all occurrences (/g means global replacement)

In [6]:
#import gene_path.csv 
path = pd.read_csv(f"{datadir}/gene_path_case.csv", names=['ID', 'WGT'])
path.head

<bound method NDFrame.head of                       ID                                     WGT
0      ENST00000001146.6    ./weights/ENST00000001146.6.wgt.RDat
1      ENST00000004103.8    ./weights/ENST00000004103.8.wgt.RDat
2     ENST00000008180.13   ./weights/ENST00000008180.13.wgt.RDat
3     ENST00000009041.11   ./weights/ENST00000009041.11.wgt.RDat
4     ENST00000011989.11   ./weights/ENST00000011989.11.wgt.RDat
...                  ...                                     ...
9477   ENST00000673312.1    ./weights/ENST00000673312.1.wgt.RDat
9478   ENST00000673427.1    ./weights/ENST00000673427.1.wgt.RDat
9479   ENST00000673452.1    ./weights/ENST00000673452.1.wgt.RDat
9480   ENST00000673471.1    ./weights/ENST00000673471.1.wgt.RDat
9481   ENST00000673487.1    ./weights/ENST00000673487.1.wgt.RDat

[9482 rows x 2 columns]>

In [10]:
#remove .hsq rows --> didn't need in this
#path = path[~path.WGT.str.contains(".hsq")]
#path.head

<bound method NDFrame.head of                        ID                                     WGT
0       ENST00000001146.6    ./weights/ENST00000001146.6.wgt.RDat
1       ENST00000004103.8    ./weights/ENST00000004103.8.wgt.RDat
2      ENST00000008180.13   ./weights/ENST00000008180.13.wgt.RDat
3      ENST00000009041.11   ./weights/ENST00000009041.11.wgt.RDat
4      ENST00000011989.11   ./weights/ENST00000011989.11.wgt.RDat
...                   ...                                     ...
11576   ENST00000673312.1    ./weights/ENST00000673312.1.wgt.RDat
11577   ENST00000673427.1    ./weights/ENST00000673427.1.wgt.RDat
11578   ENST00000673452.1    ./weights/ENST00000673452.1.wgt.RDat
11579   ENST00000673471.1    ./weights/ENST00000673471.1.wgt.RDat
11580   ENST00000673487.1    ./weights/ENST00000673487.1.wgt.RDat

[11581 rows x 2 columns]>

In [11]:
#save path as csv -- run only once for download
#path.to_csv(r'./data_folder/gene_path_case.csv' ,index=False)

In [7]:
#merge chr_pos_df and path
merged = chr_pos_df.merge(path, how='inner', on='ID')
merged.head

<bound method NDFrame.head of                      ID  CHR         P0         P1  \
0     ENST00000629041.1    1   45581219   45581321   
1     ENST00000470459.6    1  161222296  161223628   
2     ENST00000470472.5    1   35176378   35190574   
3     ENST00000417869.6    1   39522287   39556662   
4     ENST00000647711.1    1  109711780  109715240   
...                 ...  ...        ...        ...   
9477  ENST00000426108.1   22   36178554   36179385   
9478  ENST00000609499.5   22   42118347   42124899   
9479  ENST00000472589.1   22   42554909   42555966   
9480  ENST00000472575.5   22   20965177   20973619   
9481  ENST00000426113.5   22   20967248   20974096   

                                        WGT  
0      ./weights/ENST00000629041.1.wgt.RDat  
1      ./weights/ENST00000470459.6.wgt.RDat  
2      ./weights/ENST00000470472.5.wgt.RDat  
3      ./weights/ENST00000417869.6.wgt.RDat  
4      ./weights/ENST00000647711.1.wgt.RDat  
...                                     ...  

In [8]:
#check chr 
chr_list = merged['CHR'].unique().tolist()
chr_list

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]

In [9]:
#drop ./weights/ from WGT
merged['WGT'] = merged['WGT'].str.replace('./weights/', '')
merged.head

/usr/local/Anaconda/envs/py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


<bound method NDFrame.head of                      ID  CHR         P0         P1  \
0     ENST00000629041.1    1   45581219   45581321   
1     ENST00000470459.6    1  161222296  161223628   
2     ENST00000470472.5    1   35176378   35190574   
3     ENST00000417869.6    1   39522287   39556662   
4     ENST00000647711.1    1  109711780  109715240   
...                 ...  ...        ...        ...   
9477  ENST00000426108.1   22   36178554   36179385   
9478  ENST00000609499.5   22   42118347   42124899   
9479  ENST00000472589.1   22   42554909   42555966   
9480  ENST00000472575.5   22   20965177   20973619   
9481  ENST00000426113.5   22   20967248   20974096   

                              WGT  
0      ENST00000629041.1.wgt.RDat  
1      ENST00000470459.6.wgt.RDat  
2      ENST00000470472.5.wgt.RDat  
3      ENST00000417869.6.wgt.RDat  
4      ENST00000647711.1.wgt.RDat  
...                           ...  
9477   ENST00000426108.1.wgt.RDat  
9478   ENST00000609499.5.wgt.RDat

In [10]:
#rearrange columns and save the file as synapse.pos
output = merged[['WGT', 'ID', 'CHR', 'P1', 'P0']]
output.to_csv(r'./data_folder/synapse_case.pos', index=False, sep="\t")

In [ ]:
#check the RDat file sizes - check few files from above in terminal: the file sizes are ranging 500-1200
#ls -l filename   #Displays Size of the specified file
#ls -l *          #Displays Size of All the files in the current directory
#ls -al *         #Displays Size of All the files including hidden files in the current directory
#ls -al dir/      #Displays Size of All the files including hidden files in the 'dir' directory

!ls -l ENST00000426108.1.wgt.RDat 

In [20]:
#set paths
sumstat_path = f'{datadir}/meta.txt'
weights_path = f'{datadir}/synapse_case.pos'
weights_dir = f'{basedir}/output/weights_case/'
fusiondir = f'/data/songy4/twas/fusion_twas'
fusion_ldref_basename = f'{fusiondir}/LDREF/1000G.EUR'
fusion_assoc_script = f'{fusiondir}/FUSION.assoc_test.R'

!mkdir --parents output/pd_case

sumstat = pd.read_csv(sumstat_path, sep='\t')

##check if weights_dir should be f'{basedir}/output/weights'####

In [23]:
#fusion calculation -- take about 3 min each chromosome 
for i in range(1, 23):
    PD_OUT = f'output/pd_case/PD_case.{i}.dat'
    fusion_cmd_i = f'\
    Rscript {fusion_assoc_script} \
    --sumstats {sumstat_path} \
    --weights {weights_path} \
    --weights_dir {weights_dir} \
    --ref_ld_chr {fusion_ldref_basename}. \
    --chr {i} \
    --out {PD_OUT}'
    shell_do(fusion_cmd_i)

Executing: Rscript /data/songy4/twas/fusion_twas/FUSION.assoc_test.R --sumstats /data/songy4/tes/data_folder/meta.txt --weights /data/songy4/tes/data_folder/synapse_case.pos --weights_dir /data/songy4/tes/output/weights_case/ --ref_ld_chr /data/songy4/twas/fusion_twas/LDREF/1000G.EUR. --chr 1 --out output/pd_case/PD_case.1.dat
Executing: Rscript /data/songy4/twas/fusion_twas/FUSION.assoc_test.R --sumstats /data/songy4/tes/data_folder/meta.txt --weights /data/songy4/tes/data_folder/synapse_case.pos --weights_dir /data/songy4/tes/output/weights_case/ --ref_ld_chr /data/songy4/twas/fusion_twas/LDREF/1000G.EUR. --chr 2 --out output/pd_case/PD_case.2.dat
Executing: Rscript /data/songy4/twas/fusion_twas/FUSION.assoc_test.R --sumstats /data/songy4/tes/data_folder/meta.txt --weights /data/songy4/tes/data_folder/synapse_case.pos --weights_dir /data/songy4/tes/output/weights_case/ --ref_ld_chr /data/songy4/twas/fusion_twas/LDREF/1000G.EUR. --chr 3 --out output/pd_case/PD_case.3.dat
Executing: Rs

# running list of questions
1. what about non-autosome transcripts? i.e. MT
2. how many snps in common between ref and geno?
3. how many snps per transcript should we expect (0-500?)


In [44]:
p_in = 'tmp/ENSG00000188976.tmp.cv'
f"gemma -miss 1 -maf 0 -r2 1 -rpace 1000 -wpace 1000 -bfile {p_in} -bslmm 2 -o ENSG00000188976"

'gemma -miss 1 -maf 0 -r2 1 -rpace 1000 -wpace 1000 -bfile tmp/ENSG00000188976.tmp.cv -bslmm 2 -o ENSG00000188976'